In [1]:
import json
import os

import pandas as pd

In [2]:
task = 'wow'  # Change this: options = [cnn_dailymail, wow]

In [3]:
eval_results_path = '../eval_results'

files = os.listdir(eval_results_path)
files = [os.path.join(eval_results_path, fname) for fname in files]

results = {}
for fname in files:
    if not fname.endswith('.json'):
        continue

    if task not in fname:
        continue
    with open(fname, 'r') as f:
        data = json.load(f)
        results[os.path.splitext(fname)[0].replace(eval_results_path + '/', '').replace(
            'unieval_', '').replace(f'-DecoderDisc-{task}', '')] = data

In [4]:
mfma = results.pop(f'{task}-MFMA', None)

df = pd.DataFrame(results).T

if mfma is not None:
    mfma = {os.path.basename(k): v for k, v in mfma.items()}
    mfma_df = pd.DataFrame(mfma).T
    df = df.merge(mfma_df, left_index=True, right_index=True)

if task == 'wow':
    df['prob.'] = [-1] * len(df)  # get these later!


In [5]:
# pip install tabulate

if task == 'wow':
    # TODO: column names for unieval dialogue
    columns = ['KF1', 'K-Copy', 'F1', 'BLEU', 'RougeL',  'chrF', 'meteor', "naturalness", "coherence", "groundedness", "understandability", 'prob.']
    # for order
    rows = ['baseline-wow-openai_gpt-3.5-turbo',  # zeroshot
            'baseline-wow-openai_text-davinci-003',
            'baseline-wow-google-flan-t5-xl',
            'baseline-wow-google-flan-t5-xxl',
            'baseline-wow-bigscience-T0pp',
            'baseline-wow-google-flan-t5-xl-sft',  # sft
            'fudge-wow-google-flan-t5-xl-fudge-RAND',  # guided decoding baseline
            'nado-wow-google-flan-t5-xl-nado-ALL-v2-alpha0.25',
            'ppl_mcts-wow-google-flan-t5-xl-RAND',
            'fudge-wow-google-flan-t5-xl-fudge-RIPA',  # KWD
            'ppl_mcts-wow-google-flan-t5-xl-RIPA',  # KCTS
            ]
else:
    columns = ['KF1', 'K-Copy', 'F1', 'BLEU', 'RougeL', 'chrF', 'meteor', 'coherence', 'consistency', 'fluency',  'relevance']
    rows = ['baseline-cnn_dailymail-openai_gpt-3.5-turbo',  # zeroshot
            'baseline-cnn_dailymail-openai_text-davinci-003',
            'baseline-cnn_dailymail-google-flan-t5-xxl',
            'baseline-cnn_dailymail-bigscience-T0pp',
            'baseline-cnn_dailymail-google-flan-t5-xl',
            'fudge-cnn_dailymail-google-flan-t5-xl-fudge-RAND',  # guided decoding baseline
            'nado-cnn_dailymail-google-flan-t5-xl-nado-ALL-v2-alpha0.25',
            'ppl_mcts-cnn_dailymail-google-flan-t5-xl-RAND',
            'fudge-cnn_dailymail-google-flan-t5-xl-fudge-RIPA',  # KWD
            "ppl_mcts-cnn_dailymail-google-flan-t5-xl-RIPA",  # KCTS
            ]

out_df = (df.loc[rows, columns] * 100).round(2)
out_df['chrF'] = (out_df['chrF'] / 100).round(2)
display(out_df)
print('markdown')
print(out_df.to_markdown())
print()
print('latex')
print(out_df.to_latex(na_rep=''))

,KF1,K-Copy,F1,BLEU,RougeL,chrF,meteor,naturalness,coherence,groundedness,understandability,prob.
baseline-wow-openai_gpt-3.5-turbo,49.41,39.71,30.32,6.91,26.24,34.95,31.67,57.62,96.41,96.15,57.27,-100
baseline-wow-openai_text-davinci-003,25.91,28.22,22.32,3.01,18.70,27.86,23.06,42.77,98.07,92.42,42.51,-100
baseline-wow-google-flan-t5-xl,34.50,37.07,21.18,6.81,19.64,24.88,18.53,71.69,82.21,75.70,70.90,-100
baseline-wow-google-flan-t5-xxl,28.20,32.33,19.11,5.53,17.55,24.15,17.16,72.37,84.24,75.51,71.68,-100
baseline-wow-bigscience-T0pp,26.94,28.80,17.57,4.13,16.14,19.84,13.37,52.79,85.26,70.14,52.27,-100
baseline-wow-google-flan-t5-xl-sft,39.85,37.79,28.08,9.41,25.11,31.17,25.40,76.44,92.36,95.16,75.83,-100
fudge-wow-google-flan-t5-xl-fudge-RAND,55.30,54.04,29.43,11.72,27.35,31.50,26.00,73.68,88.20,83.53,72.90,-100
nado-wow-google-flan-t5-xl-nado-ALL-v2-alpha0.25,50.20,50.10,27.86,10.57,26.01,29.84,24.51,74.14,88.35,81.10,73.34,-100
ppl_mcts-wow-google-flan-t5-xl-RAND,55.54,54.21,29.56,11.69,27.48,31.60,26.08,74.54,88.16,83.90,73.76,-100
fudge-wow-google-flan-t5-xl-fudge-RIPA,58.19,56.56,30.71,12.74,28.27,33.40,28.10,68.44,90.51,87.86,67.84,-100


markdown
|                                                  |   KF1 |   K-Copy |    F1 |   BLEU |   RougeL |   chrF |   meteor |   naturalness |   coherence |   groundedness |   understandability |   prob. |
|:-------------------------------------------------|------:|---------:|------:|-------:|---------:|-------:|---------:|--------------:|------------:|---------------:|--------------------:|--------:|
| baseline-wow-openai_gpt-3.5-turbo                | 49.41 |    39.71 | 30.32 |   6.91 |    26.24 |  34.95 |    31.67 |         57.62 |       96.41 |          96.15 |               57.27 |    -100 |
| baseline-wow-openai_text-davinci-003             | 25.91 |    28.22 | 22.32 |   3.01 |    18.7  |  27.86 |    23.06 |         42.77 |       98.07 |          92.42 |               42.51 |    -100 |
| baseline-wow-google-flan-t5-xl                   | 34.5  |    37.07 | 21.18 |   6.81 |    19.64 |  24.88 |    18.53 |         71.69 |       82.21 |          75.7  |               70.9  |    -10

/tmp/ipykernel_45523/474750440.py:40: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(out_df.to_latex(na_rep=''))
